In [1]:
#geostatspy est un package GSLIB(Geostatistical Libraries) écrit en fortran par (Deustch et Journel ,1992),et reécrit en langage python 
#par Michael pyrcz, projet lancé en 2017
import geostatspy.GSLIB as GSLIB          # GSLIB utilité, visualisation 
import geostatspy.geostats as geostats    # GSLIB methods convert to Python

In [2]:
import numpy as np                        # ndarrys pour les données gridded 
import pandas as pd                       # DataFrames pour les données tabulaires
import os                                 # répertoire courant
import matplotlib.pyplot as plt           # affichage
import seaborn as sns                     # affichage
import warnings                           # supprimé les warnings de seaborn pairplot
from sklearn.model_selection import train_test_split # train / test DatFrame split
from __future__ import print_function
from ipywidgets import *
import ipywidgets as widgets
%matplotlib 

Using matplotlib backend: <object object at 0x00000284D0C77870>


In [3]:
#Keras et tensorflow sont des modules avancés des réseaux de neurones, ils permettent de réaliser 
#le Deep Learning en utilisant des packages bien définis
import tensorflow as tf
tf.__version__                            # Version installée

'2.9.1'

In [4]:
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
import keras
from tensorflow.python.keras import backend as k

In [5]:
os.chdir("D:/PI") #repertoire courant

In [6]:

df = pd.read_csv('Perenco_data2.csv', delimiter=';')     # importation de la base des données
df.head()

,Xcoord,Ycoord,Reservoirs,Avg Porosity,Avg Perm,AI
0,433200,478810,5,0.220761,140.021266,3413.063944
1,433200,478310,5,0.237154,39.837129,3074.562617
2,433200,477810,5,0.234352,84.992437,2292.783358
3,433200,477310,5,0.244553,90.632307,2494.848885
4,433200,476810,5,0.231787,811.547979,2522.063995


In [7]:
df['Reservoirs level']=np.zeros(len(df));
df['Reservoirs level']= np.where(df['Reservoirs']==5, 'S4', 'S5')

In [8]:
df.head()

,Xcoord,Ycoord,Reservoirs,Avg Porosity,Avg Perm,AI,Reservoirs level
0,433200,478810,5,0.220761,140.021266,3413.063944,S4
1,433200,478310,5,0.237154,39.837129,3074.562617,S4
2,433200,477810,5,0.234352,84.992437,2292.783358,S4
3,433200,477310,5,0.244553,90.632307,2494.848885,S4
4,433200,476810,5,0.231787,811.547979,2522.063995,S4


In [9]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Xcoord,480.0,436944.166667,2550.821091,432700.000000,434700.000000,436600.000000,439000.000000,442200.000000
Ycoord,480.0,475013.291667,2851.396757,470000.000000,472600.000000,475200.000000,477400.000000,479800.000000
Reservoirs,480.0,4.616667,0.486706,4.000000,4.000000,5.000000,5.000000,5.000000
Avg Porosity,480.0,0.189440,0.031702,0.117562,0.165884,0.185443,0.216513,0.261091
Avg Perm,480.0,520.932093,1226.207190,0.005776,6.539988,49.451463,369.470756,10319.904850
AI,480.0,3758.879653,779.990582,1746.387548,3212.900120,3719.883000,4236.160395,6194.573653


In [10]:
pormin = 0.10; pormax = 0.33              # valeur de porosité min and max
permmin = 0.01; permmax = 3000            # valeur de perméabilité min and max
Impmin = 1000.0; Impmax = 8000              # Impédance acoustique min and max
Rmin = 4.0; Rmax = 5.0                        # Réservoirs min and max
cmap = plt.cm.nipy_spectral                 # affichage couleur
#(nipy_spectral), hsv, gist_rainbow, gist_ncar,cubehelix, jet, terrain, (brg), gnuplot, gist_stern, Set1, Set2, Set3,
#Dark2, Paired, Spectral



In [11]:
seismic = np.loadtxt(open("ImpA.csv", "rb"), delimiter=",") # Importation des données d'impédance acosutique

In [12]:
xmin = 43200.0; xmax = 44200.0                # gamme des coordonnées x
ymin = 47000.0; ymax = 48000.0                 # gamme des des coordonnées y
ny = 100; nx = 100                        # Nombre de cellules X et Y
cell_size = 10.0;                         # dimension du grid cell 
plt.subplot(121)
GSLIB.pixelplt_st(seismic,xmin,xmax,ymin,ymax,cell_size,Impmin,Impmax,'Impédance acoustique','X coord(m*10)','Y coord(m*10)','Impédance acoustique (kg/m2s*10^3)',cmap)
plt.subplots_adjust(left=0.0, bottom=0.0, right=4.0, top=1.5, wspace=0.2, hspace=0.2)
plt.show()

In [13]:
df_subset = df.iloc[:,:]

In [14]:
df_subset.head()

,Xcoord,Ycoord,Reservoirs,Avg Porosity,Avg Perm,AI,Reservoirs level
0,433200,478810,5,0.220761,140.021266,3413.063944,S4
1,433200,478310,5,0.237154,39.837129,3074.562617,S4
2,433200,477810,5,0.234352,84.992437,2292.783358,S4
3,433200,477310,5,0.244553,90.632307,2494.848885,S4
4,433200,476810,5,0.231787,811.547979,2522.063995,S4


In [15]:
#sns.set(rc = {'figure.figsize':(4,5)})
sns.pairplot(df_subset, plot_kws = {'color':'green'})

In [16]:
sns.pairplot(df_subset, hue = 'Reservoirs')
sns.set(style="ticks")
plt.show()

In [17]:
por_min = df_subset['Avg Porosity'].min(); por_max = df_subset['Avg Porosity'].max();
AI_min = df_subset['AI'].min(); AI_max = df_subset['AI'].max();

df_subset['norm_Avg Porosity'] = (df_subset['Avg Porosity'] - por_min)/(por_max - por_min) * 2 - 1
df_subset['norm_AI'] = (df_subset['AI'] - AI_min)/(AI_max - AI_min) * 2 - 1

In [18]:
df_subset.head()

,Xcoord,Ycoord,Reservoirs,Avg Porosity,Avg Perm,AI,Reservoirs level,norm_Avg Porosity,norm_AI
0,433200,478810,5,0.220761,140.021266,3413.063944,S4,0.438033,-0.250626
1,433200,478310,5,0.237154,39.837129,3074.562617,S4,0.666461,-0.402824
2,433200,477810,5,0.234352,84.992437,2292.783358,S4,0.627403,-0.754329
3,433200,477310,5,0.244553,90.632307,2494.848885,S4,0.769557,-0.663476
4,433200,476810,5,0.231787,811.547979,2522.063995,S4,0.591663,-0.651239


In [19]:
df_subset.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Xcoord,480.0,436944.166667,2550.821091,432700.000000,434700.000000,436600.000000,439000.000000,442200.000000
Ycoord,480.0,475013.291667,2851.396757,470000.000000,472600.000000,475200.000000,477400.000000,479800.000000
Reservoirs,480.0,4.616667,0.486706,4.000000,4.000000,5.000000,5.000000,5.000000
Avg Porosity,480.0,0.189440,0.031702,0.117562,0.165884,0.185443,0.216513,0.261091
Avg Perm,480.0,520.932093,1226.207190,0.005776,6.539988,49.451463,369.470756,10319.904850
AI,480.0,3758.879653,779.990582,1746.387548,3212.900120,3719.883000,4236.160395,6194.573653
norm_Avg Porosity,480.0,0.001582,0.441746,-1.000000,-0.326658,-0.054107,0.378828,1.000000
norm_AI,480.0,-0.095140,0.350701,-1.000000,-0.340624,-0.112674,0.119455,1.000000


In [20]:
X = df_subset.iloc[:, [8]]                 # extraction des éléments de prédiction (Impédance acoustique)
Y = df_subset.iloc[:,[7]]                 # extraction des éléments de réponse ( Porosité)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=73073)

In [21]:
plt.subplot(121)
plt.plot(X_train['norm_AI'].values,Y_train['norm_Avg Porosity'].values, 'ko', markerfacecolor='green',alpha = 0.6, markeredgecolor='black', label = "Train")
plt.plot(X_test['norm_AI'].values,Y_test['norm_Avg Porosity'].values, 'ko', markerfacecolor='yellow',alpha = 0.6, markeredgecolor='black', label = "Test")
plt.legend(loc='upper right');
plt.title('Train and Test: Porosité vs. Impédance acoustique')
plt.ylabel('Porosité (Normalisé)')
plt.xlabel('Impédance acoustique (Normalisé)')
plt.xlim(-1,1); plt.ylim(-1,1);

plt.subplots_adjust(left=0.0, bottom=0.0, right=3.2, top=1.0, wspace=0.2, hspace=0.2)
plt.show()

In [22]:
#Construire le modèle réseaux de neurones pour la prédiction du modèle (input: AI - > Output: Porosity)
#fonctions d'activation
model = Sequential([
    Dense(1, activation='sigmoid', input_shape=(1,)), #  couche Input
    Dense(50, activation='relu'), #  couche 1
    Dense(100, activation='relu'), #  couche 2
    Dense(50, activation='tanh'), #  couche 3
    Dense(50, activation='linear'), #  couche 4                                     Deep Learning
    Dense(100, activation='relu'), #  couche 5
    Dense(100, activation='relu'), # couche 6
    Dense(20, activation='tanh'), #  couche 7
    Dense(1, activation='tanh') # couche output
])

In [23]:

h = widgets.Text(value = 'Deep Learning Web Dashboard for seismic attributes prediction', placeholder ='sly', description='string', disabled = False, layout=Layout(width='1080px', height='40px'))
xs = widgets.FloatSlider(min=-10, max=30, value=10, step=1, description='x', orientation='horizontal', continuous_update=True )
xs.style.handle_color='green'
xr = widgets.FloatSlider(min=-10, max=30, value=10, step=1, description='x', orientation='horizontal', continuous_update=True )
xr.style.handle_color='orange'
xt = widgets.FloatSlider(min=-10, max=30, value=10, step=1, description='x', orientation='horizontal', continuous_update=True )
xt.style.handle_color='purple'
xl = widgets.FloatSlider(min=-10, max=30, value=10, step=1, description='x', orientation='horizontal', continuous_update=True )
xl.style.handle_color='orange'


def sigmoide(xs):
    s = 1/(1+np.exp(-xs))
    plt.figure(figsize=(5,3))
    plt.title('Fonction Sigmoide', fontsize = 15)
    plt.xlabel('Input')
    plt.ylabel('Output')
    plt.xlim(-10.0, 10.0)
    plt.ylim(-1.5, 1.5)
    x=np.linspace(-10, 10, 100)
    y=1/(1+np.exp(-x))
    plt.plot(x, y,'-', color='green' )
    plt.plot(xs,s,'o', markerfacecolor='green', markeredgecolor='black', markersize=20, alpha=0.4)
    plt.grid(True)
    plt.show()   

def tanh(xt):
    t = np.tanh(xt)
    plt.figure(figsize=(5,3))
    plt.title('Fonction tanh', fontsize = 15)
    plt.xlabel('Input')
    plt.ylabel('Output')
    plt.xlim(-10.0, 10.0)
    plt.ylim(-1.5, 1.5)
    x=np.linspace(-10, 10, 100)
    y=np.tanh(x)
    plt.plot(x, y,'-', color='purple' )
    plt.plot(xt,t,'o', markerfacecolor='purple', markeredgecolor='black', markersize=20, alpha=0.4)
    plt.grid(True)
    plt.show()

def relu(xr):
    if xr<0:
        r=0
    else:
        r=xr
    
    plt.figure(figsize=(5,3))
    plt.title('Fonction ReLU', fontsize = 15)
    plt.xlabel('Input')
    plt.ylabel('Output')
    plt.xlim(-1.0, 1.0)
    plt.ylim(-1.5, 1.5)
    x=np.linspace(-1.0, 1.0, 100)
    zero=np.zeros(len(x))
    y=np.max([zero, x], axis=0)
    plt.plot(x, y,'-', color='orange' )
    plt.plot(xr,r,'o', markerfacecolor='orange', markeredgecolor='black', markersize=20, alpha=0.4)
    plt.grid(True)
    plt.show()
w=widgets.Dropdown(options=['tanh','sigmoide','relu'],  description='fonction activation:')
c=widgets.Checkbox(description='Checkbox', value=True)


interact(tanh, xt =10.0)
interact(sigmoide, xs =10.0)
interact(relu, xr=1.0)





interactive(children=(FloatSlider(value=10.0, description='xt', max=30.0, min=-10.0), Output()), _dom_classes=…

interactive(children=(FloatSlider(value=10.0, description='xs', max=30.0, min=-10.0), Output()), _dom_classes=…

interactive(children=(FloatSlider(value=1.0, description='xr', max=3.0, min=-1.0), Output()), _dom_classes=('w…

<function __main__.relu(xr)>

In [24]:
sgd = tf.keras.optimizers.SGD( learning_rate=0.01, momentum=0.0, nesterov=False, name='SGD') # algorithme du gradient descendant stochastique
#adam = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam') # algorithme d'
#rms = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.95, momentum=0.0, epsilon=1e-07, centered=False,name='RMSprop')
#adagrad = tf.keras.optimizers.Adagrad(learning_rate=0.001, initial_accumulator_value=0.1, epsilon=1e-07,name='Adagrad')
#nadam = tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name='Nadam')
#adamax = tf.keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9 beta_2=0.999,epsilon=1e-07, name='Adamax')
#adadelta = tf.keras.optimizers.Adadelta(learning_rate=0.001,rho=0.95, epsilon=1e-07, name='Adadelta')
#ftrl = tf.keras.optimizers.Ftrl(learning_rate=0.001,learning_rate_power=-0.5, initial_accumulator_value=0.1, l1_regularization_strength=0.0,l2_regularization_strength=0.0, name='Ftrl', l2_shrinkage_regularization_strength=0.0,beta=0.0)

In [25]:
model.compile(optimizer='sgd', loss='huber', metrics = ['accuracy']) #Computes the logarithm of the hyperbolic cosine of the prediction error. 
#log_cosh, mape, mse, msle, MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError, MeanSquaredLogarithmicError, mean_squared_error
#mean_absolute_error, mean_absolute_percentage_error, mean_squared_logarithmic_error, cosine_similarity
# huber, log_cosh functions

In [26]:
hist = model.fit(X_train, Y_train,
          batch_size=32, epochs=400,
          validation_data=(X_test, Y_test))

Epoch 1/400
12/12 [==============================] - 15s 128ms/step - loss: 0.0980 - accuracy: 0.0000e+00 - val_loss: 0.0983 - val_accuracy: 0.0000e+00
Epoch 2/400
12/12 [==============================] - 0s 30ms/step - loss: 0.0975 - accuracy: 0.0000e+00 - val_loss: 0.0982 - val_accuracy: 0.0000e+00
Epoch 3/400
12/12 [==============================] - 0s 19ms/step - loss: 0.0976 - accuracy: 0.0000e+00 - val_loss: 0.0982 - val_accuracy: 0.0000e+00
Epoch 4/400
12/12 [==============================] - 0s 16ms/step - loss: 0.0974 - accuracy: 0.0000e+00 - val_loss: 0.0983 - val_accuracy: 0.0000e+00
Epoch 5/400
12/12 [==============================] - 0s 20ms/step - loss: 0.0974 - accuracy: 0.0000e+00 - val_loss: 0.0983 - val_accuracy: 0.0000e+00
Epoch 6/400
12/12 [==============================] - 0s 17ms/step - loss: 0.0975 - accuracy: 0.0000e+00 - val_loss: 0.0983 - val_accuracy: 0.0000e+00
Epoch 7/400
12/12 [==============================] - 0s 22ms/step - loss: 0.0975 - accuracy: 0.000

12/12 [==============================] - 0s 15ms/step - loss: 0.0974 - accuracy: 0.0000e+00 - val_loss: 0.0982 - val_accuracy: 0.0000e+00
Epoch 110/400
12/12 [==============================] - 0s 12ms/step - loss: 0.0973 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 111/400
12/12 [==============================] - 0s 14ms/step - loss: 0.0973 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 112/400
12/12 [==============================] - 0s 17ms/step - loss: 0.0977 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 113/400
12/12 [==============================] - 0s 17ms/step - loss: 0.0973 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 114/400
12/12 [==============================] - 0s 17ms/step - loss: 0.0973 - accuracy: 0.0000e+00 - val_loss: 0.0982 - val_accuracy: 0.0000e+00
Epoch 115/400
12/12 [==============================] - 0s 16ms/step - loss: 0.0974 - accuracy: 0.0000e

Epoch 163/400
12/12 [==============================] - 0s 12ms/step - loss: 0.0973 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 164/400
12/12 [==============================] - 0s 16ms/step - loss: 0.0974 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 165/400
12/12 [==============================] - 0s 12ms/step - loss: 0.0974 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 166/400
12/12 [==============================] - 0s 13ms/step - loss: 0.0974 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 167/400
12/12 [==============================] - 0s 14ms/step - loss: 0.0973 - accuracy: 0.0000e+00 - val_loss: 0.0982 - val_accuracy: 0.0000e+00
Epoch 168/400
12/12 [==============================] - 0s 12ms/step - loss: 0.0973 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 169/400
12/12 [==============================] - 0s 13ms/step - loss: 0.0974 - acc

Epoch 217/400
12/12 [==============================] - 0s 18ms/step - loss: 0.0973 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 218/400
12/12 [==============================] - 0s 22ms/step - loss: 0.0973 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 219/400
12/12 [==============================] - 0s 19ms/step - loss: 0.0973 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 220/400
12/12 [==============================] - 0s 12ms/step - loss: 0.0973 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 221/400
12/12 [==============================] - 0s 13ms/step - loss: 0.0973 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 222/400
12/12 [==============================] - 0s 11ms/step - loss: 0.0974 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 223/400
12/12 [==============================] - 0s 13ms/step - loss: 0.0973 - acc

Epoch 271/400
12/12 [==============================] - 0s 14ms/step - loss: 0.0974 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 272/400
12/12 [==============================] - 0s 16ms/step - loss: 0.0974 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 273/400
12/12 [==============================] - 0s 15ms/step - loss: 0.0972 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 274/400
12/12 [==============================] - 0s 21ms/step - loss: 0.0972 - accuracy: 0.0000e+00 - val_loss: 0.0980 - val_accuracy: 0.0000e+00
Epoch 275/400
12/12 [==============================] - 0s 16ms/step - loss: 0.0974 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 276/400
12/12 [==============================] - 0s 20ms/step - loss: 0.0974 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 277/400
12/12 [==============================] - 0s 22ms/step - loss: 0.0973 - acc

Epoch 325/400
12/12 [==============================] - 0s 10ms/step - loss: 0.0972 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 326/400
12/12 [==============================] - 0s 11ms/step - loss: 0.0972 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 327/400
12/12 [==============================] - 0s 11ms/step - loss: 0.0973 - accuracy: 0.0000e+00 - val_loss: 0.0980 - val_accuracy: 0.0000e+00
Epoch 328/400
12/12 [==============================] - 0s 13ms/step - loss: 0.0973 - accuracy: 0.0000e+00 - val_loss: 0.0980 - val_accuracy: 0.0000e+00
Epoch 329/400
12/12 [==============================] - 0s 14ms/step - loss: 0.0973 - accuracy: 0.0000e+00 - val_loss: 0.0980 - val_accuracy: 0.0000e+00
Epoch 330/400
12/12 [==============================] - 0s 11ms/step - loss: 0.0973 - accuracy: 0.0000e+00 - val_loss: 0.0980 - val_accuracy: 0.0000e+00
Epoch 331/400
12/12 [==============================] - 0s 13ms/step - loss: 0.0973 - acc

Epoch 379/400
12/12 [==============================] - 0s 13ms/step - loss: 0.0972 - accuracy: 0.0000e+00 - val_loss: 0.0980 - val_accuracy: 0.0000e+00
Epoch 380/400
12/12 [==============================] - 0s 10ms/step - loss: 0.0973 - accuracy: 0.0000e+00 - val_loss: 0.0980 - val_accuracy: 0.0000e+00
Epoch 381/400
12/12 [==============================] - 0s 11ms/step - loss: 0.0973 - accuracy: 0.0000e+00 - val_loss: 0.0980 - val_accuracy: 0.0000e+00
Epoch 382/400
12/12 [==============================] - 0s 13ms/step - loss: 0.0973 - accuracy: 0.0000e+00 - val_loss: 0.0980 - val_accuracy: 0.0000e+00
Epoch 383/400
12/12 [==============================] - 0s 12ms/step - loss: 0.0972 - accuracy: 0.0000e+00 - val_loss: 0.0981 - val_accuracy: 0.0000e+00
Epoch 384/400
12/12 [==============================] - 0s 10ms/step - loss: 0.0973 - accuracy: 0.0000e+00 - val_loss: 0.0980 - val_accuracy: 0.0000e+00
Epoch 385/400
12/12 [==============================] - 0s 9ms/step - loss: 0.0972 - accu

In [27]:
#evalt = model.evaluate(x=None,y=None,batch_size=None,verbose=1,sample_weight=None,steps=None,)
#print(model.evaluate(X_test, Y_test))

In [28]:
print(model.metrics_names)

['loss', 'accuracy']


In [29]:

plt.subplot(2,1,1)
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Modèle du Loss function')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.grid(True)
plt.tight_layout(pad=1.0, h_pad=1.0)

In [30]:
AIbins = np.linspace(df_subset['AI'].values.min(), df_subset['AI'].values.max(), 30) # paramètrer le nombre de bins prediction
norm_AIbins = (AIbins-AI_min)/(AI_max-AI_min)*2-1 # normalisation

pred_norm_porsity = model.predict(np.array(norm_AIbins)) # prédiction du réseau de neurone

pred_porosity = (pred_norm_porsity + 1)/2*(por_max - por_min)+por_min # backtransform

plt.figure()
plt.plot(df_subset['AI'].values,df_subset['Avg Porosity'].values,'ko', alpha = 0.8)
plt.plot(AIbins,pred_porosity,'r-',linewidth=5)
plt.xlabel('Impédance acoustique (kg/m2s*10^6)')
plt.ylabel('Porosité (fraction)')
plt.show()

1/1 [==============================] - 0s 429ms/step


In [31]:

norm_seismic = (seismic-AI_min)/(AI_max-AI_min)*2-1

pred_norm_porosity_map = model.predict(np.array(norm_seismic.flatten())) #  prédiction de neurone 

pred_porosity_map = (pred_norm_porosity_map + 1)/2*(por_max - por_min)+por_min # Backtransform à l'intervalle des valeurs
pred_porosity_map = np.reshape(pred_porosity_map,(ny,nx))

plt.subplot(121)
GSLIB.pixelplt_st(norm_seismic,xmin,xmax,ymin,ymax,cell_size,-1,1,'Impédance acoustique','X(m)','Y(m)','Impédance acoustique(normalisé)',cmap)

plt.subplot(122)
GSLIB.pixelplt_st(pred_porosity_map,xmin,xmax,ymin,ymax,cell_size,por_min,por_max,'Porosité','X(m)','Y(m)','Porosité',cmap)

plt.subplots_adjust(left=0.0, bottom=0.0, right=3.0, top=1.2, wspace=0.02, hspace=1.0)
plt.show()

313/313 [==============================] - 1s 4ms/step


In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 2         
                                                                 
 dense_1 (Dense)             (None, 50)                100       
                                                                 
 dense_2 (Dense)             (None, 100)               5100      
                                                                 
 dense_3 (Dense)             (None, 50)                5050      
                                                                 
 dense_4 (Dense)             (None, 50)                2550      
                                                                 
 dense_5 (Dense)             (None, 100)               5100      
                                                                 
 dense_6 (Dense)             (None, 100)               1

In [33]:
# Weights of Model

for layer in model.layers:
    g = layer.get_config()
    h = layer.get_weights()
    print(g)
    print(h)
    print('\n')

{'name': 'dense', 'trainable': True, 'batch_input_shape': (None, 1), 'dtype': 'float32', 'units': 1, 'activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
[array([[0.08110295]], dtype=float32), array([-0.00033434], dtype=float32)]


{'name': 'dense_1', 'trainable': True, 'dtype': 'float32', 'units': 50, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
[array([[-0.05205238,  0.23547526,  0.07505696,  0.18066789,  0.07947917,
         0.32314792, -0.26748553,  0.281

{'name': 'dense_8', 'trainable': True, 'dtype': 'float32', 'units': 1, 'activation': 'tanh', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
[array([[-0.24598299],
       [-0.46490762],
       [ 0.34007454],
       [ 0.01191493],
       [-0.0436655 ],
       [-0.21908386],
       [ 0.34043047],
       [-0.22561419],
       [-0.4341976 ],
       [-0.26398963],
       [-0.18490718],
       [-0.2321551 ],
       [ 0.28557482],
       [ 0.5232321 ],
       [-0.3613776 ],
       [-0.47087064],
       [-0.20843145],
       [-0.25570187],
       [-0.08375023],
       [ 0.30904406]], dtype=float32), array([0.00393426], dtype=float32)]




In [34]:
from tkinter import *
from tkinter import filedialog

In [35]:
win=Tk()
canvas=Canvas()
canvas.pack()
photo=PhotoImage('D:/PI/perenco.png')
canvas.create_image(0,0, image=photo, anchor=NW)
def Open():
    win.filename=filedialog.askopenfilename(filetypes=(('PDF Files', '*.pdf'), ('Doc Files', '*.docx'), 
                                                       ('Text Files', '*.txt'), ('All Files', '*.*')))

In [36]:
menuBar=Menu(win)
menuFile=Menu(menuBar)
menuEdit=Menu(menuBar)
menuView=Menu(menuBar)
menuInsert=Menu(menuBar)
menuHelp=Menu(menuBar)

In [37]:
menuBar.add_cascade(label='File', menu=menuFile)
menuBar.add_cascade(label='Edit', menu=menuEdit)
menuBar.add_cascade(label='View', menu=menuView)
menuBar.add_cascade(label='Insert', menu=menuInsert)
menuBar.add_cascade(label='Help', menu=menuHelp)

menuFile.add_command(label='New')
menuFile.add_command(label='Open', command=Open)
menuFile.add_command(label='Save')
menuFile.add_command(label='Save as')
menuFile.add_command(label='Quit', command=quit)


win.config(menu=menuBar)
win.mainloop()